In [1]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, AutoConfig
import os
from datasets import load_from_disk
os.environ['TRANSFORMERS_CACHE'] = '../transformers_cache'



In [5]:
data_test = load_from_disk("../datasets/gsm8k/test")
data_test


Dataset({
    features: ['question', 'answer'],
    num_rows: 1319
})

In [4]:
# data_train = load_from_disk("../datasets/gsm8k/train")
# data_train

In [6]:
import pandas as pd

In [60]:
path = '../outputs/gsm8k/LLaMA8B/generated_outputs_test.json'
df = pd.read_json(path)

In [61]:
df.head()

,input,output
0,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...
1,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...
2,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...
3,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...
4,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...


In [62]:
def get_answer_dataset(row):
    return row['answer'].split('####')[-1]
def get_answer_model(row):
    return row['output'].split('####')[-1]

In [63]:
# df.drop(columns=['score'],inplace=True)
df['model_answer']=df.apply(get_answer_model,axis=1)

In [64]:
df.head()

,input,output,model_answer
0,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,18
1,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,3
2,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,"120,000"
3,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,540
4,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,20


In [65]:
from evaluate import load
from tqdm import tqdm
exact_match_metric = load("exact_match")

In [66]:
# results = exact_match_metric.compute(predictions=[df.iloc[3]['model_answer']], references=[data_feedback["answer"][3].split("####")[-1]])

In [67]:
test = pd.DataFrame(data_test)
test.head()

,question,answer
0,Janet’s ducks lay 16 eggs per day. She eats th...,Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eg...
1,A robe takes 2 bolts of blue fiber and half th...,It takes 2/2=<<2/2=1>>1 bolt of white fiber\nS...
2,Josh decides to try flipping a house. He buys...,The cost of the house and repairs came out to ...
3,James decides to run 3 sprints 3 times a week....,He sprints 3*3=<<3*3=9>>9 times\nSo he runs 9*...
4,"Every day, Wendi feeds each of her chickens th...","If each chicken eats 3 cups of feed per day, t..."


In [68]:
test['GT_Answer']=test.apply(get_answer_dataset,axis=1)

In [69]:
test.head()

,question,answer,GT_Answer
0,Janet’s ducks lay 16 eggs per day. She eats th...,Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eg...,18
1,A robe takes 2 bolts of blue fiber and half th...,It takes 2/2=<<2/2=1>>1 bolt of white fiber\nS...,3
2,Josh decides to try flipping a house. He buys...,The cost of the house and repairs came out to ...,70000
3,James decides to run 3 sprints 3 times a week....,He sprints 3*3=<<3*3=9>>9 times\nSo he runs 9*...,540
4,"Every day, Wendi feeds each of her chickens th...","If each chicken eats 3 cups of feed per day, t...",20


In [70]:
for i in tqdm(range(df.shape[0])):
    df.loc[i,'score']=exact_match_metric.compute(predictions=[df.iloc[i]['model_answer']], references=[test.iloc[i]["GT_Answer"]])['exact_match']

100%|██████████| 1319/1319 [00:34<00:00, 38.08it/s]


In [71]:
df.head()

,input,output,model_answer,score
0,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,18,1.0
1,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,3,1.0
2,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,"120,000",0.0
3,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,540,1.0
4,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,20,1.0


In [72]:
score= exact_match_metric.compute(predictions=df['model_answer'].tolist(), references=test["GT_Answer"].tolist())

In [73]:
score

{'exact_match': 0.7877179681576952}

In [74]:
# df['score'] = (df['model_answer'] == feedback['GT_Answer']).astype(int)

In [75]:
# from tqdm import tqdm
# for i in tqdm(range(10)):
    

In [76]:
df.head(10)

,input,output,model_answer,score
0,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,18,1.0
1,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,3,1.0
2,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,"120,000",0.0
3,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,540,1.0
4,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,20,1.0
5,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,64,1.0
6,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,260,1.0
7,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,80,0.0
8,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,315,0.0
9,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,460,1.0


In [77]:
df.drop(columns=['model_answer'],inplace=True)

In [78]:
df.head()

,input,output,score
0,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,1.0
1,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,1.0
2,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,0.0
3,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,1.0
4,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,1.0


In [79]:
# df.to_json("/scratch/workspace/wenlongzhao_umass_edu-analyze//Bidirectional-Teacher-Student-Communication-for-LLM-Alignment/outputs/gsm8k/LLaMA8B/generated_outputs1.json", orient='records', lines=True)

In [80]:
records = df.to_dict(orient='records')

In [81]:
import json
formatted_json = json.dumps(records, indent=4)


In [82]:
with open("../outputs/gsm8k/LLaMA8B/generated_outputs_test_1.json", "w") as f:
    f.write(formatted_json)

In [83]:
df['score'].sum()

1039.0

In [84]:
df['score'].sum()/1319

0.7877179681576952

In [33]:
df.shape[0]

1319